In [7]:
VERSION = '001'
BATCHSIZE = 32
MAX_EPOCHS = 50
BACKBONE = "resnet50"
MODEL_NAME = "ResNet50_transfer"
NUM_WORKERS = 31

In [8]:
import torch
from torchvision import transforms
import torch.nn as nn
import os
import pytorch_lightning as pl  # Add this import
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from code.classifier import PneumoniaClassifier, Config
from code.trainer import PneumoniaTrainer
from code.dataloader import PneumoniaDataset
from code.custom_checkpoint import CustomModelCheckpoint
from code.project_globals import TEST_DIR, TRAIN_DIR, VAL_DIR


In [9]:
# Enable CUDA Launch Blocking
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Set float32 matrix multiplication precision to 'medium' to utilize Tensor Cores
torch.set_float32_matmul_precision('medium')

NameError: name 'os' is not defined

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Sun Nov 17 16:48:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   45C    P8             14W /  285W |    5154MiB /  12282MiB |     14%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
# Example usage
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainer = PneumoniaTrainer(
    version='001',
    batch_size=32,
    max_epochs=50,
    backbone="resnet50",
    model_name="ResNet50_transfer",
    num_workers=31,
    transform=transform,
    weight_decay=1e-4,
    dropout=0.5
)

trainer.train()


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Exception in thread Exception ignored in: <function _ConnectionBase.__del__ at 0x0000015E8761BBA0>
Traceback (most recent call last):
  File "C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\multiprocessing\connection.py", line 133, in __del__
QueueFeederThread:
Traceback (most recent call last):
  File "C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\multiprocessing\queues.py", line 259, in _feed
    self._close()
  File "C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\multiprocessing\connection.py", line 282, in _close
    reader_close()
  File "C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\multiprocessing\connection.py", line 178, in close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid
    self._close()
  File "C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\multiprocessing\connection.py", line 282, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid

During handling of the above excepti

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
# Add Dropout to the safe globals list
torch.serialization.add_safe_globals([nn.Dropout, Config])

# Load the checkpoint with weights_only=True
checkpoint_path = '../checkpoints/ResNet50_transfer-epoch=08-val_loss=0.20_v001.ckpt'
checkpoint = torch.load(checkpoint_path, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

In [12]:

# Load the TensorBoard extension
%load_ext tensorboard

# Start TensorBoard and point it to the log directory
%tensorboard --logdir tb_logs/ResNet50_transfer/version_1

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 29500), started 0:01:37 ago. (Use '!kill 29500' to kill it.)

# Add Regularization